In [0]:
import requests
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import pandas as pd
import time
import math
import re
from tqdm import tnrange

In [0]:
#path = "D:/chromedriver.exe" 

In [0]:
driver = webdriver.Chrome(path)

In [0]:
#정규표현식을 이용하여 문자열에서 마지막 숫자 찾기
def get_last_number(string): 
    regex = re.compile('(\d+)(?!.*\d)') # 찾고자 하는 패턴의 정규표현식 컴파일 
    matching = regex.findall(string) # 패턴과 매칭되는 문자를 리스트로 반환 
    if matching: 
        # 매칭된 문자 리스트가 있다면 리스트 반환 
        return ''.join(matching)
    else: # 없다면 fasle를 반환 
        print('매칭되는 패턴이 없습니다.') 
        return False

In [0]:
n=1407       #페이지에 따라 수정
page = math.ceil(n / 20)

link_list=list()
title = list()
question = list()
category = list()
answer = list()
num = list()

#질문 페이지 url받아오기
for i in tnrange(1,page+1):
    #페이지에 따라 수정
    url = 'https://kin.naver.com/userinfo/answerList.nhn?u=NyO3RCKxH0MrMufO6%2Bs%2Bp20T7PWxM%2BUdTgrB%2BiHTPe8%3D&page=' + str(i) 
#    print(url)
    driver.get(url)
    element = driver.find_elements_by_xpath("""//*[(@id = "au_board_list")]//*[contains(concat( " ", @class, " " ), concat( " ", "title", " " ))]//a""")
    for link in element:
        a=str(link.get_attribute("href"))
        link_list.append(a)
#         time.sleep(0.5)


for i in tnrange(0,len(link_list)):
    #####질문######
    driver.get(link_list[i])
    time.sleep(0.5)
    title.append("".join(driver.find_element_by_css_selector(".title").text))
    category.append("".join(driver.find_element_by_css_selector('.tag-list__item--category').text[10:]))
    time.sleep(0.5)
    #본문이 없는 경우 예외처리
    try:
        question.append("".join(driver.find_element_by_css_selector("#content .c-heading__content").text))
        time.sleep(0.5)
    except:
        question.append('NA')
        
    ######찾는 답변의 위치######
    num.append(get_last_number(link_list[i]))
    
    ######답변#######
    #driver.get(link_list[i])
    try:
        tmp = driver.find_element_by_css_selector("#answer_"+ num[i])
        time.sleep(0.5)
        tmp.text.strip()
        answer.append(tmp.text.strip())
    except:
        element = driver.find_element_by_css_selector('#nextPageButton')
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        time.sleep(0.5)
        a = soup.select('div[id=answer_'+num[i]+']')
        time.sleep(0.5)
        try:
            a[0].get_text()
            answer.append(a[0].get_text())
        except:
            element2 = driver.find_element_by_css_selector('#nextPageButton')
            driver.execute_script("arguments[0].click();", element2)
            time.sleep(0.5)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            b = soup.select('div[id=answer_'+num[i]+']')
            time.sleep(0.5)
            try:
                b[0].get_text()
                answer.append(b[0].get_text())
            except:
                element3 = driver.find_element_by_css_selector('#nextPageButton')
                driver.execute_script("arguments[0].click();", element3)
                time.sleep(0.5)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                c = soup.select('div[id=answer_'+num[i]+']')
                time.sleep(0.5)
                c[0].get_text()
                answer.append(c[0].get_text())

In [0]:
df = pd.DataFrame({"title" : title, "category": category ,"question" : question, "answer" : answer, "url" : link_list})